In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('ticks', {"axes.linewidth": "1",'axes.yaxis.grid':False})

In [2]:
import os
import pickle as pk
import numpy as np
import pandas as pd
from phys.phys_utils import nnan_pearsonr as nnan_pearsonr
from phys import data_utils, phys_utils

from rnn_analysis import utils as analysis_utils
from phys.plot_scripts import generic_plot_utils as rr_gpu
from phys.plot_scripts import decoding_summary as dec_plots
from phys.plot_scripts import rnn_comparisons_2021 as rnn_plots
from sklearn.decomposition import PCA, FastICA, FactorAnalysis

/home/rishir/envs/py36/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [3]:
figoutpath_base = '/om/user/rishir/figs/mpong_phys/paper_scratch_202202/'

In [7]:
fn = '/om/user/rishir/lib/MentalPong/phys/results/rnn_comparison_results/rnn_compare_all_hand_dmfc_occ_50ms_neural_responses_reliable_FactorAnalysis_50.pkl'
dat = pk.load(open(fn, 'rb'))

In [19]:
from scipy.stats import ttest_ind

In [15]:
dat['summary']['loss_weight_type'].unique()

array(['mov', 'sim-mov', 'vis-mov', 'vis-sim-mov'], dtype=object)

In [22]:
metricn = 'pdist_similarity_occ_end_pad0_euclidean_r_xy_n_sb'
f1, f2 = 'mov', 'sim-mov'
x0 = dat['summary'].query('loss_weight_type == "%s"' % f1)[metricn]
x1 = dat['summary'].query('loss_weight_type == "%s"' % f2)[metricn]
ttest_ind(x0, x1)

Ttest_indResult(statistic=-6.210114862663557, pvalue=1.4278861421440431e-08)

In [12]:
dat['summary']['pdist_similarity_occ_end_pad0_euclidean_r_xy_n_sb']

0      0.107102
1      0.145436
2      0.263376
3      0.261741
4      0.149182
         ...   
187    0.266530
188    0.375156
189    0.360957
190    0.272846
191    0.283752
Name: pdist_similarity_occ_end_pad0_euclidean_r_xy_n_sb, Length: 192, dtype: float64

In [4]:
def load_neural_data_base(subject_id, mask_fn, 
                          condition='occ', 
                          neural_data_to_use='neural_responses_reliable_FactorAnalysis_20'):
    neural_data = data_utils.load_neural_dataset(subject_id=subject_id, timebinsize=50, recompute_augment=False)

    condition = 'occ'
    beh_to_decode = ['ball_pos_x_TRUE', 'ball_pos_y_TRUE', 'ball_pos_dx_TRUE', 'ball_pos_dy_TRUE',
                    't_from_start', 't_from_occ', 't_from_end']

    mask_fn_neur = mask_fn_beh = mask_fn
    mask_neur_cxt = np.array(neural_data['masks'][condition][mask_fn_neur])
    mask_beh_cxt = np.array(neural_data['masks'][condition][mask_fn_beh])

    data_neur_nxcxt = np.array(neural_data[neural_data_to_use][condition])
    data_beh_bxcxt = np.array([neural_data['behavioral_responses'][condition][fk] for fk in beh_to_decode])

    data = {
        'data_neur_nxcxt': data_neur_nxcxt,
        'data_beh_bxcxt': data_beh_bxcxt,
        'mask_neur_cxt': mask_neur_cxt,
        'mask_beh_cxt': mask_beh_cxt,
    }
    return data

In [5]:
data = load_neural_data_base('all_hand_dmfc', 'start_end_pad0')

In [6]:
data['data_neur_nxcxt']

dict_keys(['data_neur_nxcxt', 'data_beh_bxcxt', 'mask_neur_cxt', 'mask_beh_cxt'])